In [ ]:
import scipy as sp

from sklearn.datasets import load_digits as load

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# This lines are used to remove warning from scikit-learn when QDA is used, to prevent noisy verbosity in the notebook: should not be used in practice!
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load data
X, y = load(return_X_y=True) # load_iris(return_X_y=True)
print(X.shape)
print(X.max())

# Standardize data
sc = MinMaxScaler()
X = sc.fit_transform(X) # Scale data between 0 and 1

# The stratification ensures that the proportion of each class from the orginal data is preserved in the train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.40, random_state=0, stratify=y)

# Define the classifier list
classifiers = [["SVM", SVC()], 
               ["KNN", KNeighborsClassifier()],
               ["QDA", QuadraticDiscriminantAnalysis()], 
               ["LDA", LinearDiscriminantAnalysis(solver="lsqr")],
               ["RF",  RandomForestClassifier()]]

# Hyperparameters range
gamma = 2.0**sp.arange(-4,2) # Scale of the RBF kernel
C = 10.0**sp.arange(-1,4) # Penality of the optimization problem

# Define the dictionnary of parameters to optimize
param_grids = [dict(kernel=['rbf'], gamma=2.0**sp.arange(-4,2), C=10.0**sp.arange(0,4)), # Scale and regularization
               dict(n_neighbors = sp.arange(1,40)), # number of neighbors for KNN
               dict(reg_param = sp.linspace(0,0.1,30)), # Regularization parameter for QDA
               dict(shrinkage = sp.linspace(0,0.5,30)), # Regularization parameter for LDA
               dict(n_estimators=sp.arange(20, 150, 10)) # Number of trees
               ]

for classifier, param_grid in zip(classifiers, param_grids):
    grid = GridSearchCV(classifier[1], param_grid=param_grid, cv= 3, n_jobs=-1)
    grid.fit(X_train, y_train)
    
    clf = grid.best_estimator_ 
    clf.fit(X_train,y_train)
    
    y_pred = clf.predict(X_test)
    print("Classification accuracy for {}: {:.2f} (best parameters {})".format(classifier[0],accuracy_score(y_test,y_pred),grid.best_params_))
